# 파이토치에서 여러 모델을 하나의 파일에 저장하기 불러오기

`GAN`이나 `sequence-to-sequence`, 앙상블 모델(ensemble of models)과 같이 여러 `torch.nn.Module`로 구성된 모델을 저장할 때는 각 모델의 state_dict와 해당 옵티마이저(optimizer)의 사전을 저장해야 한다.

## 1. 데이터 불러올 때 필요한 라이브러리들 불러오기

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

## 2. 신경망을 구성하고 초기화하기

In [9]:
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.conv1 = nn.Conv2d(3, 6, 5)
    self.conv2 = nn.Conv2d(6, 16, 5)
    
    self.pool = nn.MaxPool2d(2, 2)
    
    self.fc1 = nn.Linear(16 * 5 * 5, 120)
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, 10)
    
  def forward(self, x):
    x = self.pool(F.relu(self.conv1(x)))
    x = self.pool(F.relu(self.conv2(x)))
    x = x.view(-1, 16 * 5 * 5)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x
  
netA = Net()
netB = Net()

## 3. 옵티마이저 초기화하기

In [10]:
optimizerA = optim.SGD(netA.parameters(), lr=0.001, momentum=0.9)
optimizerB = optim.SGD(netB.parameters(), lr=0.001, momentum=0.9)

## 4. 여러 모델을 저장하기

In [12]:
PATH = "model.pt"
torch.save({
    'modelA_state_dict': netA.state_dict(),
    'modelB_state_dict': netB.state_dict(),
    'optimizerA_state_dict': optimizerA.state_dict(),
    'optimizerB_state_dict': optimizerB.state_dict(),
}, PATH)

## 5. 여러 모델들 불러오기

In [14]:
import torch.nn as nn
modelA = Net()
modelB = Net()

optimModelA = optim.SGD(modelA.parameters(), lr=0.1, momentum=0.9)
optimModelA = optim.SGD(modelB.parameters(), lr=0.1, momentum=0.9)

checkpoint = torch.load(PATH)

modelA.load_state_dict(checkpoint['modelA_state_dict'])
modelB.load_state_dict(checkpoint['modelB_state_dict'])
optimizerA.load_state_dict(checkpoint['optimizerA_state_dict'])
optimizerB.load_state_dict(checkpoint['optimizerB_state_dict'])

modelA.eval()
modelB.eval()


Net(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)